In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import svm
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
import lightgbm as lgbm
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import statistics

# Здесь были исследования (можно не смотреть)


In [6]:
feats = pd.read_csv('/content/players_feats.csv')
train = pd.read_csv('/content/train.csv')

feats['who_win'] = 0
map_names = feats['map_name'].unique()
print(map_names)
map_dict = {
    'Ancient': 0, 
    'Dust2': 1, 
    'Mirage': 2,
    'Inferno': 3,
    'Nuke': 4,
    'Overpass':5,
    'Vertigo': 6
}
feats['map_name'] = feats['map_name'].map(map_dict).fillna(7)

for match in train.iterrows():
  match_info = list(match[1])
  team2_id = int(match_info[2])
  map_id = int(match_info[0])
  who_win = int(train[train['map_id'] == map_id]['who_win'])
  if who_win == 1:
    feats.loc[(feats['map_id'] == map_id) & (feats['team_id'] == team2_id), 'who_win'] = 1

X = feats[np.isfinite(feats).all(1)]
y = X['who_win'].values
maps_id = X['map_id']
teams_id = X['team_id']
X1 = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X1)
df = pd.DataFrame(x_scaled)
df.columns = feats.columns
X_new = df.drop(['who_win','p1_id','p2_id','p3_id','p4_id','p5_id'], axis = 1)
X_new['map_id'] = maps_id
X_new['team_id'] = teams_id

# X_new['map_id'].replace('', np.nan, inplace=True)
# X_new.dropna(subset=['map_id'], inplace=True)
# y = X_new['who_win'].values
# X_new = X_new.drop(['who_win'], axis = 1)
print(len(y))
print(len(X_new))

# X_new.drop(X_new[X_new.map_id.value_counts() == 1].index, inplace=True)

# sns.pairplot(data=X_new, diag_kind='kde')
# X_new.to_excel('tr1.xlsx')

['Ancient' 'Dust2' 'Mirage' 'Inferno' 'Nuke' 'Overpass' 'Vertigo']
1470
1470


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_new,y, test_size = 0.2, random_state = 42)

#Create a XGBoost Regressor
reg = xgb.XGBRegressor(objective="reg:linear", random_state=42)

# reg = RandomForestRegressor()

# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# reg = svm.SVR()

# # Train the model using the training sets 
reg.fit(X_train, y_train)

# # Model prediction on train data
y_pred = reg.predict(X_train)

[03:05:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
def boost_models(x):
    #transforming target variable through quantile transformer
    regr_trans = TransformedTargetRegressor(regressor=x, transformer=QuantileTransformer(output_distribution='normal'))
    regr_trans.fit(X_train, y_train)
    yhat = regr_trans.predict(X_test)
    algoname= x.__class__.__name__
    return algoname, round(metrics.r2_score(y_test, yhat),3), round(metrics.mean_absolute_error(y_test, yhat),2), round(np.sqrt(metrics.mean_squared_error(y_test, yhat)),2), round(metrics.roc_auc_score(y_test, yhat),2)

algo=[GradientBoostingRegressor(), lgbm.LGBMRegressor(), xgb.XGBRFRegressor()]
score=[]
for a in algo:
    score.append(boost_models(a))

pd.DataFrame(score, columns=['Model', 'Score', 'MAE', 'RMSE', 'ROC-AUC'])

In [ ]:
param_grid = {'n_estimators': [100, 80, 60, 55, 51, 45],  
              'max_depth': [7, 8],
              'reg_lambda' :[0.26, 0.25, 0.2]
             }
                
grid = GridSearchCV(xgb.XGBRFRegressor(), param_grid, refit = True, verbose = 3, n_jobs=-1) #
regr_trans = TransformedTargetRegressor(regressor=grid, transformer=QuantileTransformer(output_distribution='normal'))

# fitting the model for grid search 
grid_result=regr_trans.fit(X_train, y_train)
best_params=grid_result.regressor_.best_params_
print(best_params)

#using best params to create and fit model
best_model = xgb.XGBRFRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], reg_lambda=best_params["reg_lambda"])
regr_trans = TransformedTargetRegressor(regressor=best_model, transformer=QuantileTransformer(output_distribution='normal'))
regr_trans.fit(X_train, y_train)
yhat = regr_trans.predict(X_test)

#evaluate metrics
metrics.r2_score(y_test, yhat), metrics.mean_absolute_error(y_test, yhat), np.sqrt(metrics.mean_squared_error(y_test, yhat)), metrics.roc_auc_score(y_test, yhat)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[12:25:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'max_depth': 7, 'n_estimators': 100, 'reg_lambda': 0.2}
[12:25:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-0.2930555555555556,
 0.19387755102040816,
 0.44031528592635544,
 0.5081018518518517)

In [40]:
# Model Evaluation
acc_ROC_AUC_train= metrics.roc_auc_score(y_train, y_pred)
acc_R2_train= metrics.r2_score(y_train, y_pred)
acc_AR2_train= 1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
acc_MAE_train= metrics.mean_absolute_error(y_train, y_pred)
acc_MSE_train= metrics.mean_squared_error(y_train, y_pred)
acc_RMSE_train= np.sqrt(metrics.mean_squared_error(y_train, y_pred))

print('R^2:',acc_R2_train)
print('Adjusted R^2:', acc_AR2_train)
print('MAE:', acc_MAE_train)
print('MSE:', acc_MSE_train)
print('RMSE:', acc_RMSE_train)
print("ROC-AUC:", acc_ROC_AUC_train)

R^2: 0.9999999956684433
Adjusted R^2: 0.9999999951573938
MAE: 2.6306124771533368e-05
MSE: 7.621667469484052e-10
RMSE: 2.7607367620771185e-05
ROC-AUC: 1.0


In [ ]:
test = pd.read_csv('/content/test.csv')
y_test_pred = reg.predict(test)

print(X_test)

X_test['aaa'] = y_test_pred

# X_test.to_excel('aaaTest.xlsx')

In [9]:
# Model Evaluation
acc_ROC_AUC_train= metrics.roc_auc_score(y_test, y_test_pred)
acc_R2_train= metrics.r2_score(y_test, y_test_pred)
acc_AR2_train= 1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_train.shape[1]-1)
acc_MAE_train= metrics.mean_absolute_error(y_test, y_test_pred)
acc_MSE_train= metrics.mean_squared_error(y_test, y_test_pred)
acc_RMSE_train= np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print('R^2:',acc_R2_train)
print('Adjusted R^2:', acc_AR2_train)
print('MAE:', acc_MAE_train)
print('MSE:', acc_MSE_train)
print('RMSE:', acc_RMSE_train)
print("ROC-AUC:", acc_ROC_AUC_train)

R^2: -0.10516416367574166
Adjusted R^2: -0.904782940923484
MAE: 0.3326640230457799
MSE: 0.16570558055946144
RMSE: 0.40706950335226716
ROC-AUC: 0.5722993827160494


# Результат получен здесь

In [21]:
# Подгружаем данные
feats = pd.read_csv('/content/players_feats.csv') 
train = pd.read_csv('/content/train.csv')

# Приводим к цифрому виду названия карт
map_dict = {
    'Ancient': 0, 
    'Dust2': 1, 
    'Mirage': 2,
    'Inferno': 3,
    'Nuke': 4,
    'Overpass':5,
    'Vertigo': 6
}
feats['map_name'] = feats['map_name'].map(map_dict).fillna(7)

# Пропущенные значения, заменяем на 0, возможно, имеет смысл вообще удалить строки с нулевыми значениями или зменить например на среднее 
feats.fillna(0, inplace = True)

# Здесь будем хранить значения вида Комнда/карта: уровнь скилла на этой карте
grp = {}

# Добавляем столбец с меткой результата матча 
feats['who_win'] = 0

# Заполняем стоблец с результатами матча, согласно данным из train.csv
for match in train.iterrows():
  match_info = list(match[1])
  team2_id = int(match_info[2])
  map_id = int(match_info[0])
  who_win = int(train[train['map_id'] == map_id]['who_win'])
  if who_win == 1:
    feats.loc[(feats['map_id'] == map_id) & (feats['team_id'] == team2_id), 'who_win'] = 1

# Сохраняем значения столбцов, которые мы не хотим нормировать
team_id = feats['team_id']
map_name = feats['map_name']

# Нормируем значения (значения варьируются от 0 до 1)
X = feats.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
df = pd.DataFrame(x_scaled)

df.columns = feats.columns
df['team_id'] = team_id
df['map_name'] = map_name

# Функция на вход получает алгоритм Регресси и данные, на которых обучается и делает предсказания 
# Предсказания округляем
def boost_models(x,tmp_X, tmp_y,X_test):
    regr_trans = TransformedTargetRegressor(regressor=x, transformer=QuantileTransformer(output_distribution='normal'))
    regr_trans.fit(tmp_X, tmp_y)
    yhat = regr_trans.predict(X_test)
    return yhat.mean()

# Усредняем все значения, тем самым получаем средний показатель скилла конкретной команды, на конкретной карте 
def get_score(tmp_X, tmp_y,X_test):
  algo=[GradientBoostingRegressor(), lgbm.LGBMRegressor(), xgb.XGBRFRegressor()]
  score = []
  for a in algo:
    score.append(boost_models(a,tmp_X, tmp_y,X_test))
  return np.mean(score)

# Получаем значения конкретной команды на определенной карте 
for feat in df.iterrows():
  feat = list(feat[1])
  team_id = int(feat[-4])
  map_name = int(feat[-3])

  comb_id = str(team_id) + "/" + str(map_name)
  tmp_X = feats[(feats['team_id'] == team_id) & (feats['map_name'] == map_name)]
  tmp_y = tmp_X["who_win"]
  print(tmp_X.shape)
  # Здесь я использовал два варианта: 
  # 1) с разбиением на тренировачный и тестовый (тогда приходится отбрасывать те значения, где команда сыграла 1 или 2 игры на определенной карте)
  # 2) Использование всех данных в качестве определения скила (есть риск переобучиться)
  if tmp_X.shape[0] < 2:
    # X_train, X_test, y_train, y_test = train_test_split(tmp_X, tmp_y, random_state = 42)
    # grp[comb_id] = get_score(X_train, y_train, X_test)
    continue
  else:
    X_train, X_test, y_train, y_test = train_test_split(tmp_X, tmp_y, test_size = 0.2, random_state = 42)
    grp[comb_id] = get_score(X_train, y_train, X_test)
  
print(grp)





(15, 129)
[11:52:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(7, 129)
[11:52:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(10, 129)
[11:52:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(16, 129)


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (12). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (12). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (12). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (5). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (5). n_quantiles 

[11:52:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(6, 129)
[11:52:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(7, 129)
[11:52:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(14, 129)


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (11). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (11). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (6). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (6). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (6). n_quantiles i

[11:52:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(8, 129)
[11:52:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(21, 129)
[11:52:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(16, 129)


...

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (8). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (8). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (6). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (6). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (6). n_quantiles is 

(8, 129)
[11:52:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(15, 129)
[11:52:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(10, 129)
[11:52:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (8). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (8). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (12). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (12). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (12). n_quantiles 

(16, 129)
[11:52:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
(16, 129)


KeyboardInterrupt: ignored

In [19]:
test = pd.read_csv("/content/test.csv")
predicts = []

for game in test.iterrows():
  game_info = list(game[1])
  team1_id = game_info[2]
  team2_id = game_info[3]
  map_name = map_dict[game_info[4]]
  team1_skill = grp[str(team1_id)+"/"+str(map_name)]
  team2_skill = grp[str(team2_id)+"/"+str(map_name)]
  if team1_skill > team2_skill:
    print(f"T1 = {team1_skill}, T2 = {team2_skill}. Predict = 0")
    predicts.append(0)
  elif team1_skill < team2_skill:
    print(f"T1 = {team1_skill}, T2 = {team2_skill}. Predict = 1")
    predicts.append(1)
  else:
    # Если значения скилла равны, я выставляю 0, но надо вводить какие-то показатели на основе которых будет решаться что делать в такой ситуации
    print(f"T1 = {team1_skill}, T2 = {team2_skill}. Predict = хм")
    predicts.append(0)

test['who_win'] = predicts

test.to_csv("predict.csv")

T1 = 0.0, T2 = 0.08838822974429857. Predict = 1
T1 = 0.18181818452748386, T2 = 0.49999999999999994. Predict = 1
T1 = 0.03450023673042587, T2 = 0.49999999999999994. Predict = 1
T1 = 0.12121212301832257, T2 = 0.0. Predict = 0
T1 = 0.22222222553359136, T2 = 0.8255547838659324. Predict = 1
T1 = 0.12005530508809077, T2 = 0.200000003973643. Predict = 1
T1 = 0.24242424603664514, T2 = 0.050403142234286914. Predict = 0
T1 = 0.200000003973643, T2 = 0.0463803037769363. Predict = 0
T1 = 0.10256410447450785, T2 = 0.26666666865348815. Predict = 1
T1 = 0.12698412816675883, T2 = 0.0. Predict = 0
T1 = 0.07062924216357526, T2 = 0.0. Predict = 0
T1 = 0.17777778241369457, T2 = 0.17777778241369457. Predict = хм
T1 = 0.0463803037769363, T2 = 0.200000003973643. Predict = 1
T1 = 0.10256410447450785, T2 = 0.09998126858141354. Predict = 0
T1 = 0.0760907918806367, T2 = 0.12121212301832257. Predict = 1
T1 = 0.200000003973643, T2 = 0.20833333333333334. Predict = 1
T1 = 0.20833333333333334, T2 = 0.23809524093355453